# Load Data

In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

train_dir = '../input/herbarium-2022-fgvc9/train_images/'
test_dir = '../input/herbarium-2022-fgvc9/test_images/'

with open("../input/herbarium-2022-fgvc9/train_metadata.json") as json_file:
    train_meta = json.load(json_file)
with open("../input/herbarium-2022-fgvc9/test_metadata.json") as json_file:
    test_meta = json.load(json_file)

# JSON -> Dataframe

In [2]:
image_ids = [image["image_id"] for image in train_meta["images"]]
image_dirs = [train_dir + image['file_name'] for image in train_meta["images"]]
category_ids = [annotation['category_id'] for annotation in train_meta['annotations']]
genus_ids = [annotation['genus_id'] for annotation in train_meta['annotations']]

test_ids = [image['image_id'] for image in test_meta]
test_dirs = [test_dir + image['file_name'] for image in test_meta]

#Create the initial training dataframe with the above defined columns
train_df = pd.DataFrame({
    "image_id" : image_ids,
    "image_dir" : image_dirs,
    "category" : category_ids,
    "genus" : genus_ids})

#Create a testing dataframe
test_df = pd.DataFrame({
    "test_id" : test_ids,
    "test_dir" : test_dirs
})

train_df

,image_id,image_dir,category,genus
0,00000__001,../input/herbarium-2022-fgvc9/train_images/000...,0,1
1,00000__002,../input/herbarium-2022-fgvc9/train_images/000...,0,1
2,00000__003,../input/herbarium-2022-fgvc9/train_images/000...,0,1
3,00000__004,../input/herbarium-2022-fgvc9/train_images/000...,0,1
4,00000__005,../input/herbarium-2022-fgvc9/train_images/000...,0,1
...,...,...,...,...
839767,15504__032,../input/herbarium-2022-fgvc9/train_images/155...,15504,2584
839768,15504__033,../input/herbarium-2022-fgvc9/train_images/155...,15504,2584
839769,15504__035,../input/herbarium-2022-fgvc9/train_images/155...,15504,2584
839770,15504__036,../input/herbarium-2022-fgvc9/train_images/155...,15504,2584


# Mapping genus and family

In [3]:
#Add a genus column to the dataframe
genus_map = {genus['genus_id'] : genus['genus'] for genus in train_meta['genera']}
train_df['genus'] = train_df['genus'].map(genus_map)

##Create a family column in the datagframe based on the genus names
    # Step 1: Create dictionary of genus -> family mapping
genus_family_map = {}
for category in train_meta["categories"]:
    genus = category['genus']
    family = category['family']
    genus_family_map[genus] = family

    # Step 2: Create new column with default value of None™
train_df['family'] = None

    # Step 3: Update values in new column based on genus -> family mapping
for i, row in train_df.iterrows():
    genus = row['genus']
    if genus in genus_family_map:
        family = genus_family_map[genus]
        train_df.at[i, 'family'] = family

train_df

,image_id,image_dir,category,genus,family
0,00000__001,../input/herbarium-2022-fgvc9/train_images/000...,0,Abies,Pinaceae
1,00000__002,../input/herbarium-2022-fgvc9/train_images/000...,0,Abies,Pinaceae
2,00000__003,../input/herbarium-2022-fgvc9/train_images/000...,0,Abies,Pinaceae
3,00000__004,../input/herbarium-2022-fgvc9/train_images/000...,0,Abies,Pinaceae
4,00000__005,../input/herbarium-2022-fgvc9/train_images/000...,0,Abies,Pinaceae
...,...,...,...,...,...
839767,15504__032,../input/herbarium-2022-fgvc9/train_images/155...,15504,Zygophyllum,Zygophyllaceae
839768,15504__033,../input/herbarium-2022-fgvc9/train_images/155...,15504,Zygophyllum,Zygophyllaceae
839769,15504__035,../input/herbarium-2022-fgvc9/train_images/155...,15504,Zygophyllum,Zygophyllaceae
839770,15504__036,../input/herbarium-2022-fgvc9/train_images/155...,15504,Zygophyllum,Zygophyllaceae


# Filtering to Poaceae

In [4]:
#Filter only the images of plants that are in the Poaceae family
train_df = train_df.loc[train_df['family'] == 'Poaceae']
#Reset index
train_df = train_df.reset_index(drop=True)
train_df

,image_id,image_dir,category,genus,family
0,00333__001,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae
1,00333__002,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae
2,00333__003,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae
3,00333__004,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae
4,00333__005,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae
...,...,...,...,...,...
53542,15501__101,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae
53543,15501__103,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae
53544,15501__105,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae
53545,15501__106,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae


# Adding species column

In [5]:
"""#Add category_id and species column
train_df["species"] = None

# Extract category_id and species values from categories where the family is Poaceae
species_list = []
for category in train_meta["categories"]:
    if category["family"] == "Poaceae":
        species_list.append({
            "category": category["category_id"],
            "species": category["species"]
        })

# loop through data frame and species list to update species column
for i, row in train_df.iterrows():
    for species in species_list:
        if row['category'] == species['category']:
            train_df.at[i, 'species'] = species['species']"""

'#Add category_id and species column\ntrain_df["species"] = None\n\n# Extract category_id and species values from categories where the family is Poaceae\nspecies_list = []\nfor category in train_meta["categories"]:\n    if category["family"] == "Poaceae":\n        species_list.append({\n            "category": category["category_id"],\n            "species": category["species"]\n        })\n\n# loop through data frame and species list to update species column\nfor i, row in train_df.iterrows():\n    for species in species_list:\n        if row[\'category\'] == species[\'category\']:\n            train_df.at[i, \'species\'] = species[\'species\']'

# Data visualization

In [6]:
"""
genus_data = train_df['genus'].value_counts().head(15)
genus_data = pd.DataFrame({'Genus' : genus_data.index,
                     'values' : genus_data.values})
                     
plt.figure(figsize = (20, 10))
sns.barplot(x='values', y = 'Genus', data = genus_data , palette='summer_r')
plt.show()

From most to least: Muhlenbergia, Paspalum, Poa, Dichanthelium, Sporobolus, Eragrostis etc.
"""

"\ngenus_data = train_df['genus'].value_counts().head(15)\ngenus_data = pd.DataFrame({'Genus' : genus_data.index,\n                     'values' : genus_data.values})\n                     \nplt.figure(figsize = (20, 10))\nsns.barplot(x='values', y = 'Genus', data = genus_data , palette='summer_r')\nplt.show()\n\nFrom most to least: Muhlenbergia, Paspalum, Poa, Dichanthelium, Sporobolus, Eragrostis etc.\n"

# Filter to two genus

In [7]:
#Muhlenbergia data
muh_pas_df = train_df[(train_df['genus'] == 'Paspalum') | (train_df['genus'] == 'Muhlenbergia')]
muh_pas_df = muh_pas_df.reset_index(drop=True)
muh_pas_df

,image_id,image_dir,category,genus,family
0,09492__001,../input/herbarium-2022-fgvc9/train_images/094...,9492,Muhlenbergia,Poaceae
1,09492__003,../input/herbarium-2022-fgvc9/train_images/094...,9492,Muhlenbergia,Poaceae
2,09492__004,../input/herbarium-2022-fgvc9/train_images/094...,9492,Muhlenbergia,Poaceae
3,09492__005,../input/herbarium-2022-fgvc9/train_images/094...,9492,Muhlenbergia,Poaceae
4,09492__006,../input/herbarium-2022-fgvc9/train_images/094...,9492,Muhlenbergia,Poaceae
...,...,...,...,...,...
7347,10398__026,../input/herbarium-2022-fgvc9/train_images/103...,10398,Paspalum,Poaceae
7348,10398__029,../input/herbarium-2022-fgvc9/train_images/103...,10398,Paspalum,Poaceae
7349,10398__030,../input/herbarium-2022-fgvc9/train_images/103...,10398,Paspalum,Poaceae
7350,10398__031,../input/herbarium-2022-fgvc9/train_images/103...,10398,Paspalum,Poaceae


# Image displaying

In [8]:
def show_images(genus):
    images = train_df.loc[train_df['genus'] == genus]['image_dir'][:9]
    i = 1
    fig = plt.figure(figsize = (18, 18))
    plt.suptitle(genus, fontsize = '30')
    for image in images:
        img = cv2.imread(image)
        ax = fig.add_subplot(3, 3, i)
        ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        ax.set_axis_off()
        i += 1
    plt.show()

In [9]:
#show_images("Muhlenbergia")

# Splitting the dataset into training and validation

**2 Genuses**

In [10]:
muh_df = muh_pas_df[muh_pas_df["genus"] == "Muhlenbergia"] 
pas_df = muh_pas_df[muh_pas_df["genus"] == "Paspalum"]

#15 percent of images will be used for validation
# Muh total: 4228 --> 15% = 634
# Pas total: 3124 --> 15% = 467
muh_valid = muh_df.sample(n=634, random_state=42)
muh_train = muh_df.drop(muh_valid.index)
muh_valid = muh_valid.reset_index(drop=True)
muh_train = muh_train.reset_index(drop=True)

pas_valid = pas_df.sample(n=467, random_state=42)
pas_train = pas_df.drop(pas_valid.index)
pas_valid = pas_valid.reset_index(drop=True)
pas_train = pas_train.reset_index(drop=True)

# Merging the Muhlanbergia and Paspalum databases
muh_pas_train = pd.concat([muh_train, pas_train])
muh_pas_train = muh_pas_train.reset_index(drop=True)

muh_pas_valid = pd.concat([muh_valid, pas_valid])
muh_pas_valid = muh_pas_valid.reset_index(drop=True)

muh_pas_train["genus"].nunique()

2

In [11]:
train_df["genus"].value_counts().tail(20)

Otatea             33
Chaetium           30
Trichoneura        30
Coleanthus         28
Tetrapogon         27
Arctopoa           21
Willkommia         20
Setariopsis        20
Tuctoria           19
Neostapfia         18
Chusquea           18
Amelichloa         16
Allolepis          14
Ptilagrostis       14
Swallenia          14
Ptilagrostiella    14
Rhipidocladum      11
Dupontia           10
Kalinia            10
Barkworthia         8
Name: genus, dtype: int64

# Creating the model

In [12]:
import os
import torch
import torchvision
import numpy as np
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from sklearn import preprocessing
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision.models import resnet50, ResNet50_Weights

df = train_df

In [13]:
# Split the dataset for each class separately
train_dfs = []
val_dfs = []
for label in df['genus'].unique():
    # Filter the dataset to only include images with the current label
    label_df = df[df['genus'] == label]
    
    # Split the dataset into training and evaluation sets
    train_df, val_df = train_test_split(label_df, test_size=0.2)
    
    # Append the training and evaluation sets to their respective lists
    train_dfs.append(train_df)
    val_dfs.append(val_df)

# Concatenate the training and evaluation sets for all classes into single DataFrames
train_df = pd.concat(train_dfs)
val_df = pd.concat(val_dfs)

train_df['genus'] = pd.factorize(train_df['genus'])[0]
val_df['genus'] = pd.factorize(val_df['genus'])[0]

In [14]:
batch_size = 128
epochs = 10
IM_SIZE = 224

X_train, Y_train = train_df["image_dir"].values, train_df["genus"].values

X_val, Y_val = val_df["image_dir"].values, val_df["genus"].values

Transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Resize((IM_SIZE, IM_SIZE)),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [15]:
class GetData(Dataset):
    def __init__(self, FNames, Labels, Transform):
        self.fnames = FNames
        self.transform = Transform
        self.labels = Labels         
        
    def __len__(self):
        return len(self.fnames)

    def __getitem__(self, index):       
        x = Image.open(self.fnames[index])
    
        if "train" in self.fnames[index]:             
            return self.transform(x), self.labels[index]
        elif "test" in self.fnames[index]:            
            return self.transform(x), self.fnames[index]

                
trainset = GetData(X_train, Y_train, Transform)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)

valset = GetData(X_val, Y_val, Transform)
valloader = DataLoader(valset, batch_size=batch_size, shuffle=True)

N_Classes = train_df['genus'].nunique()

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = models.resnet152(pretrained=True)

/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

In [16]:
print(model.fc.in_features) 
print(model.fc.out_features)

for param in model.parameters():
    param.requires_grad= False
    
n_inputs = model.fc.in_features
last_layer = nn.Linear(n_inputs, N_Classes)
model.fc = last_layer
if torch.cuda.is_available():
    model.cuda()
print(model.fc.out_features)    

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters()) 

2048
1000
158


In [17]:
from tqdm import tqdm
from sklearn.metrics import f1_score

def train(trainloader, model, criterion, optimizer, scaler, device=torch.device("cpu")):
    train_acc = 0.0
    train_loss = 0.0
    y_true = []
    y_pred = []
    for images, labels in tqdm(trainloader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast(enabled=True):
            output = model(images)
            loss = criterion(output, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            acc = ((output.argmax(dim=1) == labels).float().mean())
            train_acc += acc
            train_loss += loss
            y_true += labels.cpu().numpy().tolist()
            y_pred += output.argmax(dim=1).cpu().numpy().tolist()
            
    train_f1 = f1_score(y_true, y_pred, average='weighted')
    return train_acc/len(trainloader), train_loss/len(trainloader), train_f1               

In [18]:
def evaluate(testloader, model, criterion, device=torch.device("cpu")):
    eval_acc = 0.0
    eval_loss = 0.0
    y_true = []
    y_pred = []
    for images, labels in tqdm(testloader):
        images = images.to(device)
        labels = labels.to(device)
        with torch.no_grad():
            output = model(images)
            loss = criterion(output, labels)
        acc = ((output.argmax(dim=1) == labels).float().mean())
        eval_acc += acc
        eval_loss += loss
        y_true += labels.cpu().numpy().tolist()
        y_pred += output.argmax(dim=1).cpu().numpy().tolist()
  
    eval_f1 = f1_score(y_true, y_pred, average='weighted')
    return eval_acc/len(testloader), eval_loss/len(testloader), eval_f1    

In [19]:
%%time
scaler = torch.cuda.amp.GradScaler(enabled=True)
for epoch in range(epochs):
    train_acc, train_loss, train_f1 = train(trainloader, model, criterion, optimizer, scaler, device=device)
    eval_acc, eval_loss, eval_f1 = evaluate(valloader, model, criterion, device=torch.device("cuda"))
    print("")
    print(f"Epoch {epoch + 1} | Train Acc: {train_acc*100} | Train Loss: {train_loss} | Train F1: {train_f1}")
    print(f"\t Val Acc: {eval_acc*100} | Val Loss: {eval_loss} | Val F1: {eval_f1}")
    print("===="*8)

100%|██████████| 85/85 [05:35<00:00,  3.95s/it]



Epoch 1 | Train Acc: 20.608675003051758 | Train Loss: 3.513415813446045 | Train F1: 0.1598418999142951
	 Val Acc: 25.547075271606445 | Val Loss: 3.1203560829162598 | Val F1: 0.20775677944439944


100%|██████████| 85/85 [03:51<00:00,  2.72s/it]



Epoch 2 | Train Acc: 28.69216537475586 | Train Loss: 2.9161875247955322 | Train F1: 0.25476347372479874
	 Val Acc: 27.287006378173828 | Val Loss: 2.9837615489959717 | Val F1: 0.23994211772953758


100%|██████████| 85/85 [03:52<00:00,  2.74s/it]


...


Epoch 8 | Train Acc: 40.91231155395508 | Train Loss: 2.2569172382354736 | Train F1: 0.3987839905868106
	 Val Acc: 32.80290985107422 | Val Loss: 2.722977876663208 | Val F1: 0.30159432344977705


100%|██████████| 85/85 [03:49<00:00,  2.70s/it]



Epoch 9 | Train Acc: 41.914180755615234 | Train Loss: 2.2013325691223145 | Train F1: 0.41017023245059614
	 Val Acc: 33.92543029785156 | Val Loss: 2.722827672958374 | Val F1: 0.31746760119990236


100%|██████████| 85/85 [03:58<00:00,  2.80s/it]


Epoch 10 | Train Acc: 43.16044616699219 | Train Loss: 2.1540615558624268 | Train F1: 0.42295749748986955
	 Val Acc: 32.712196350097656 | Val Loss: 2.729304552078247 | Val F1: 0.3137855142200746
CPU times: user 2h 49min 26s, sys: 4min 54s, total: 2h 54min 21s
Wall time: 3h 18min 5s
